In [ ]:
#First we are importing the libraries we are going smeelingly use. Later on, we will import other libraries if they are necessary

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os
import glob
import json
import re
import string
import networkx as nx
import gc

from IPython.utils import io
#with io.capture_output() as captured:

 # !pip install https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/releases/v0.2.4/en_core_sci_lg-0.2.4.tar.gz

from pprint import pprint
from copy import deepcopy

from collections import  Counter
from tqdm.notebook import tqdm
#LANGUAGE DETECTION
!pip install langdetect
import langdetect
from langdetect import detect
from langdetect import DetectorFactory
#NATURAL LANGUAGE PROCESSING

#STOPWORDS IN ENGLISH SPACY
from spacy.lang.en.stop_words import STOP_WORDS

#TOKENIZER
from spacy.tokens import Span

#GENSIM

import gensim
# SPACY MODEL FOR LDA visualization

import pyLDAvis
import pyLDAvis.gensim

#TENSOR FLOWS
import tensorflow_hub as hub
import tensorflow as tf

#MATCHER
from spacy.matcher import Matcher 
SKLEARN

from sklearn.metrics.pairwise import cosine_similarity

#from sklearn.cluster import DBSCAN
#os.listdir('../input/CORD-19-research-challenge/')
#with open('../input/CORD-19-research-challenge/metadata.readme', 'r') as f:
#   data = f.read()
#    print(data)

In [ ]:
os.listdir( '/kaggle/input/all-cleaned')

In [ ]:
biorxiv='/kaggle/input/all-cleaned/biroxiv_clean''

In [ ]:
# Filter papers containing all words in list
def filter_papers_word_list(word_list):
    papers_id_list = []
    for idx, paper in biorxiv.iterrows():
        if all(x in paper.text for x in word_list):
            papers_id_list.append(paper.paper_id)

    return papers_id_list

In [ ]:
pd.set_option("display.max_colwidth", 100000) # Extend the display width to prevent split functions to not cover full text

biorxiv_environment = filter_papers_word_list(["coronavirus"])
print("Papers containing coronavirus: ", len(biorxiv_environment))

biorxiv_environment = filter_papers_word_list(["environment"])
print("Papers containing environment: ", len(biorxiv_environment))

biorxiv_environmental = filter_papers_word_list(["environmental"])
print("Papers containing environmental: ", len(biorxiv_environmental))

print("Intersection of environment and environmental: ", len(set(biorxiv_environment)-(set(biorxiv_environment)-set(biorxiv_environmental))))

biorxiv_environment_transmission = filter_papers_word_list(["coronavirus", "environment", "transmission"])
print("Number of papers containing coronavirus, environment and transmission: ", len(biorxiv_environment_transmission))

Alternative filter: word lemmatization

In [ ]:
    tokens = [token.lemma_ for token in text]
    return ' '.join([token for token in tokens])

# Filter papers containing all words in list
def filter_papers_word_list_lemma(word_list):
    papers_id_list = []
    word_list_lemma = lemmatizer(nlp(str(' '.join([token for token in word_list]))))
    for idx, paper in biorxiv.iterrows():
        if all(w in lemmatizer(nlp(paper.text)) for w in word_list_lemma):
            papers_id_list.append(paper.paper_id)

    return papers_id_list

Extract the abstract section

Most scientific papers contain a Conclusion ABSTRACT, which consists on a summary of the main observations and results from the study. In order to reduce the amount of data to analyze, it may prove useful to focus on the ABSTRACT instead of performing a full search in the paper.

In [ ]:
def extract_abstract(df, papers_id_list):
    data = df.loc[df['paper_id'].isin(papers_id_list)]
    conclusion = []
    for idx, paper in data.iterrows():
        paper_text = paper.text
        if "\nAbstract\n" in paper.text:
            conclusion.append(paper_text.split('\nAbstract\n')[1])
        else:
            conclusion.append("No Abstract section")
    data['abstract'] = abstract
        
    return data

pd.reset_option('^display.', silent=True)

Let's now extract the Conclusion section from all papers containing the words "coronavirus", "environment" and "transmission":

In [ ]:
environ_trans_conclusion = extract_conclusion(biorxiv, biorxiv_environment_transmission)
environ_trans_conclusion.head(5)

study most common words

In [ ]:
pd.set_option('display.width', 100000)

def word_bar_graph_function(df,column,title):

# adapted from https://www.kaggle.com/benhamner/most-common-forum-topic-words
topic_words = [ z.lower() for y in
                   [ x.split() for x in df[column] if isinstance(x, str)]
                   for z in y]
word_count_dict = dict(Counter(topic_words))
popular_words = sorted(word_count_dict, key = word_count_dict.get, reverse = True)
popular_words_nonstop = [w for w in popular_words if w not in stopwords.words("english")]
plt.barh(range(50), [word_count_dict[w] for w in reversed(popular_words_nonstop[0:50])])
  plt.yticks([x + 0.5 for x in range(50)], reversed(popular_words_nonstop[0:50]))
plt.title(title)
plt.show()
plt.figure(figsize=(10,10)) word_bar_graph_function(environ_trans_conclusion, "conclusion", "Most common words in papers with coronavirus, environment & transmission")

We observe that some of these words are merely situational (i.e. conclusion, section, medrxiv), but others may be particularly common in the subset of papers we have filtered. For example, words like "epicenter", "humidity", "city", "distance" and "lockdown" seem particularly related to the transmission of the virus and the environmental effects, and they probably won't be that frequent in other articles.

Let's compare this case with papers containing the word "susceptibility":

In [ ]:
biorxiv_susceptibility = filter_papers_word_list(["coronavirus", "susceptibility"])
susceptibility_conclusion = extract_conclusion(biorxiv, biorxiv_susceptibility)
plt.figure(figsize=(10,10))
word_bar_graph_function(susceptibility_conclusion, "conclusion", "Most common words in papers with coronavirus and susceptibility")

As expected, in this case there is no trace of the words "epicenter", "humidity", "city", "distance" or "lockdown". Instead, we now see large frequencies for words like "reporting", "rate", "scenarios", "diagnostic" and "protocol", which are more related to the contagion suscpetibility of the population.

what commes now?

Compare lists of words. Provide a technique to verify if papers containing two different sets of lists are redundant Most common words in conclusion. Except for stopwords, extract the most common words in the Conclusion section for all papers related to coronavirus Clean common words in conclusion by topic. Compute the most common words by topic, but this time without irrelevant information

In [ ]:
def compare_papers_by_word(list1, list2):
    biorxiv_1 = filter_papers_word_list(list1)
    biorxiv_2 = filter_papers_word_list(list2)
    if len(biorxiv_2) == len(set(biorxiv_1)-(set(biorxiv_1)-set(biorxiv_2))):
        answer = "List 1 contains List2"
    else:
        answer = "Lists are different"
    return answer

list1 = ["environment"]
list2 = ["environmental"]
compare_papers_by_word(list1, list2)

Most common words in conclusion Now we need to know which are the most common words from all papers related to coronavirus, so we can substract them when plotting the most common words from papers of a given topic:

In [ ]:
biorxiv_conclusion = filter_papers_word_list(["coronavirus"])
biorxiv_coronavirus_conclusion = extract_conclusion(biorxiv, biorxiv_conclusion)

# Filter out papers without Conclusions section
biorxiv_conclusion_informed = biorxiv_coronavirus_conclusion[biorxiv_coronavirus_conclusion["conclusion"] != "No Conclusion section"]
# Split in words
list_of_words = "".join([c for c in biorxiv_conclusion_informed.conclusion]).split()
# Remove stopwords
all_text_conclusion_nonstop = [w.lower() for w in list_of_words if w not in stopwords.words("english")]
# List of 100 most common words
most_common_words_conclusion = Counter(all_text_conclusion_nonstop).most_common(100)
most_common_words_conclusion

In [ ]:
manual_filter = [('preprint', 204),
('.', 137),
 ('license', 115),
 ('medrxiv', 108),
 ('the', 107),
 ('copyright', 78),
 ('holder', 78),
 ('peer-reviewed)', 78),
 ('available', 73),
 ('(which', 72),
 ('doi:', 64),
 ('granted', 63),
 ('display', 62),
 ('it', 62),
 ('author/funder,', 59),
 ('made', 53),
 ('4.0', 52),
 ('international', 52),
 ('data', 51),
 ('perpetuity.is', 35),
 ('cc-by-nc-nd', 28),
 ('study', 26),
 ('in', 21),
 ('we', 20),
 ('figure', 19),
 ('using', 18),
 ('author/funder.', 18),
 ('biorxiv', 18),
 ('number', 17),
 ('used', 17),
 ('sars-cov-2', 17),
 ('results', 16),
 ('covid-19', 16),
 ('perpetuity.the', 15),
 ('all', 15),
 ('1', 14),
 ('without', 13),
 ('show', 13),
 ('contributed', 13),
 ('our', 12),
 ('different', 12),
 ('2019-ncov', 12),
 ('also', 11),
 ('table', 11),
 ('coronavirus', 11),
 ('information', 11),
 ('cc-by-nc', 11),
 ('cc-by', 11),
 ('rights', 10),
 ('reserved.', 10),
 ('no', 10),
 ('reuse', 10),
 ('allowed', 10),
 ('based', 10),
 ('may', 10),
 ('consent', 10),
 ('authors', 10),
 ('https://doi.org/10.1101/2020.02.07.20021139', 10),
 ('2', 9),
 ('for', 9),
 ('however,', 9),
 ('this', 9),
 ('virus', 9),
 ('might', 8),
 ('manuscript', 8),
 ('https://doi.org/10.1101/2020.02.', 8),
 ('manuscript.', 8),
 ('reading', 8)]

Clean common words in conclusion by topic Now the easy part, use the functions from previous sections to provide alist of common words in the Conclusion section of a given subtopic:

In [ ]:
# Modify slightly Paul's function to prevent irrelevant words to appear
def word_bar_graph_function_mod(df,column,title,exception):
    # adapted from https://www.kaggle.com/benhamner/most-common-forum-topic-words
    topic_words = [ z.lower() for y in
                       [ x.split() for x in df[column] if isinstance(x, str)]
                       for z in y]
    word_count_dict = dict(Counter(topic_words))
    popular_words = sorted(word_count_dict, key = word_count_dict.get, reverse = True)
    popular_words_nonstop = [w.lower() for w in popular_words if w not in stopwords.words("english")]
    popular_words_noexcep = [w.lower() for w in popular_words_nonstop if w not in exception]
    plt.barh(range(50), [word_count_dict[w] for w in reversed(popular_words_noexcep[0:50])])
    plt.yticks([x + 0.5 for x in range(50)], reversed(popular_words_noexcep[0:50]))
    plt.title(title)
    plt.show()
    
    
# Create a general function for cleaner
def common_words_conclusion_clean(word_list, exception_list):
    
    """
    word_list = list of words that should contain the paper
    exception_list = list of most common words in desired papers, in format [(word1, counter1), (word2, counter2),...]
    """

    # Papers with the word coronavirus (I've verified that it contains all papers with COVID-19 and other key words)
    biorxiv_coronavirus = filter_papers_word_list(["coronavirus"])
    print("Papers containing coronavirus: ", len(biorxiv_coronavirus))

    # Extract Conclusion from these papers, filter out papers without it
    biorxiv_coronavirus_conclusion = extract_conclusion(biorxiv, biorxiv_coronavirus)
    biorxiv_coronavirus_conclusion = biorxiv_coronavirus_conclusion[biorxiv_coronavirus_conclusion["conclusion"] != "No Conclusion section"]

    exception_words = [x[0] for x in exception_list]

    plt.figure(figsize=(10,10))
    title = "Most common words in Conclusions on papers with ", word_list
    word_bar_graph_function_mod(biorxiv_coronavirus_conclusion, "conclusion", title, exception_words)
    
    
common_words_conclusion_clean(["coronavirus", "environment", "transmission"], manual_filter)

his is a cleaner list of common words for COVID-19 papers related to environment and transmission. This could help people filter the papers they are interested in, so that there's no need to read the whole literature.

NOW COMES THE TEXT SUMMARIZATION

Word scoring. Compute the term frequency - inverse frequency document (TFIDF) score for each word in the document Split. Split the paper's text into sentences, using dot and line break as delimiters Sentence scoring. Use text rank, with the sentence score based on word scores. Filter out sentences with >50 words Summary. The summary consists on the top 10 sentences with higher score

In [ ]:
def tfidf(data):
    tfidf_vectorizer = TfidfVectorizer(analyzer='word', stop_words = 'english', sublinear_tf=True)
    matrix = tfidf_vectorizer.fit_transform(data)
    return matrix, tfidf_vectorizer

list_corpus = list(biorxiv[biorxiv.paper_id == '6d0127f985edbe088bc279865bef25a31f54a066'].text)

tfidf_matrix, tfidf_vectorizer = tfidf(list_corpus)
word_scores_df = pd.DataFrame(tfidf_matrix.toarray(), columns = tfidf_vectorizer.get_feature_names())   # extract a df with the words' scores
word_scores = dict(zip(list(word_scores_df.columns), list(word_scores_df.iloc[0])))  # convert to dict

print("Arbitrary slice of 10 words from the dictionary")
dict(list(word_scores.items())[100:110])  

complete dictionary for all words in the paper, with their respective TFIDF scores. To split the paper into sentences, we replace all line breaks (\n, \r) by dots (.), and then split the document using the dot as a delimiter. Each sentence's score is the sum of all its word's scores, effectively penalizing short sentences (which I consider a desired criteria).

In [ ]:
#Split into sentences
sentences_list = [nlp(s) for s in list(biorxiv[biorxiv.paper_id == '6d0127f985edbe088bc279865bef25a31f54a066'].text.str.replace('\n', '.').replace('\r', '.'))]
sentences_list = str(sentences_list[0]).split('.')   # Split sentences by .
sentences_scores = {}

# Define the sentence scoring function
def get_sentence_score(sentence: str, word_scores: dict):
    words = sentence.split()
    if len(words) < 50:
        score = sum([word_scores.get(w.lower(),0) for w in words])
    else:
        score=0
    return score

# Assign scores and join the top10 sentences into the final summary
for s in sentences_list:
    sentences_scores[s] = get_sentence_score(s, word_scores)
    
top10_sentences = nlargest(10, sentences_scores, key=sentences_scores.get)
top10_sentences = [s for s in top10_sentences ]
summary = ' '.join(top10_sentences)

print("Original paper size: ", len(list_corpus[0].split()))
print("Summary size: ", len(summary.split()))
print("Reduction percentage: ", 100-round(len(summary.split())/len(list_corpus[0].split())*100, 2), "%")

This method has reduced the original document to a 18.86% of the original, which considering the increasingly large number of papers in the literature, would save many time to anyone interested into reading them partially. The obtained summary is the following:

In [ ]:
# Term frequency - inverse document frequency function
def tfidf(data):
    tfidf_vectorizer = TfidfVectorizer(analyzer='word', stop_words = 'english', sublinear_tf=True)
    matrix = tfidf_vectorizer.fit_transform(data)
    return matrix, tfidf_vectorizer


# Define the sentence scoring function
def get_sentence_score(sentence: str, word_scores: dict):
    words = sentence.split()
    if len(words) < 50:
        score = sum([word_scores.get(w.lower(),0) for w in words])
    else:
        score=0
    return score


# Summary extraction function
def extract_summary(df, paper_id):

    list_corpus = list(biorxiv[biorxiv.paper_id == paper_id].text)
    tfidf_matrix, tfidf_vectorizer = tfidf(list_corpus)
    word_scores_df = pd.DataFrame(tfidf_matrix.toarray(), columns = tfidf_vectorizer.get_feature_names())   # extract a df with the words' scores
    word_scores = dict(zip(list(word_scores_df.columns), list(word_scores_df.iloc[0])))  # convert to dict

    # Split into sentences
  
    sentences_list = [nlp(s) for s in list(biorxiv[biorxiv.paper_id == paper_id].text.str.replace('\n', '.').replace('\r', '.'))]
    sentences_list = str(sentences_list[0]).split('.')   # Split sentences by .
    sentences_scores = {}

    # Assign scores and join the top10 sentences into the final summary
    for s in sentences_list:
        sentences_scores[s] = get_sentence_score(s, word_scores)

    top10_sentences = nlargest(10, sentences_scores, key=sentences_scores.get)
    top10_sentences = [s for s in top10_sentences ]
    summary = ' '.join(top10_sentences)
        
    return summary

Notice that an input DataFrame is required, in order to generate the new summary column. In case you are interested into summarizing only some papers, you can first filter these papers from the dataframe. Results of the the previous case for reports with the words "coronavirus", "environment" and "transmision" are the following:

In [ ]:
environ_trans_summary = environ_trans_conclusion.copy()
environ_trans_summary['summary'] = environ_trans_summary['paper_id'].apply(lambda x: extract_summary(environ_trans_summary, x))
environ_trans_summary.head(5)